In [45]:
# Dependencies
import pandas as pd
from bs4 import BeautifulSoup
import requests
import pymongo
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager

In [46]:
# Initialize PyMongo to work with MongoDBs
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [47]:
# Define database and collection
db = client.mars_db
collection = db.items

In [48]:
# Setup splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 89.0.4389
[WDM] - Get LATEST driver version for 89.0.4389
[WDM] - Get LATEST driver version for 89.0.4389


[WDM] - Trying to download new driver from https://chromedriver.storage.googleapis.com/89.0.4389.23/chromedriver_mac64.zip
[WDM] - Driver has been saved in cache [/Users/melissa/.wdm/drivers/chromedriver/mac64/89.0.4389.23]


In [49]:
# URL of page to be scraped
# This is not the given website, but I get returns, so I'm working with it for now
news_url = ("https://redplanetscience.com/#")
browser.visit(news_url)

In [50]:
# HTML Object
html = browser.html

# Parse HTML with Beautiful Soup
soup = BeautifulSoup(html, 'html.parser')

In [51]:
# Retrieve the latest element that contains news title and news_paragraph
title = soup.find('div', class_='content_title').text
article_teaser = soup.find('div', class_='article_teaser_body').text

# Display scrapped data 
print(title)
print(article_teaser)

# I think my issue was using find_all and .text together incorrectly and also not having the splinter set up.
# I'm still unsure about how it all relates and why it won't work without the other, but at least I can start moving forward

NASA's Perseverance Rover Will Look at Mars Through These 'Eyes'
A pair of zoomable cameras will help scientists and rover drivers with high-resolution color images.


In [66]:
# JPL Mars image

jpl_url = ("https://spaceimages-mars.com/")
browser.visit(jpl_url)

# Set up new instance of HTML object
html = browser.html

# Parse the HTML with BeautifulSoup
soup = BeautifulSoup(html, 'html.parser')

response = soup.find('a', class_='showimg')

# print(response)

href = response['href']

img_url = jpl_url + href

img_url


'https://spaceimages-mars.com/image/featured/mars3.jpg'

Mars Facts
https://galaxyfacts-mars.com/


Visit the Mars Facts webpage here and use Pandas to scrape the table containing facts about the planet including Diameter, Mass, etc.


Use Pandas to convert the data to a HTML table string.

In [67]:
# Let's try some table reading
# Look into weird character issues
facts_url = 'https://galaxyfacts-mars.com/'

tables = pd.read_html(facts_url)
tables

mars_facts = tables[0]
mars_facts

mars_html = mars_facts.to_html('mars_facts.html')
mars_html

Mars Hemispheres


Visit the astrogeology site here to obtain high resolution images for each of Mars's hemispheres.


You will need to click each of the links to the hemispheres in order to find the image url to the full resolution image.


Save both the image url string for the full resolution hemisphere image, and the Hemisphere title containing the hemisphere name. Use a Python dictionary to store the data using the keys img_url and title.


Append the dictionary with the image url string and the hemisphere title to a list. This list will contain one dictionary for each hemisphere.

In [98]:
# Collect the image urls for the given website
hemisphere_url = 'https://marshemispheres.com/'
browser.visit(hemisphere_url)

html = browser.html

# Parse the HTML with BeautifulSoup
soup = BeautifulSoup(html, 'html.parser')

response = soup.find('a', class_='itemLink')
response

<a class="itemLink product-item" href="cerberus.html"><img alt="Cerberus Hemisphere Enhanced thumbnail" class="thumb" src="images/39d3266553462198bd2fbc4d18fbed17_cerberus_enhanced.tif_thumb.png"/></a>

In [ ]:
for x in range(1, 6):

    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')

    quotes = soup.find_all('span', class_='text')
    # This pulls only the active page

    for quote in quotes:
        print('page:', x, '-------------')
        print(quote.text)

    browser.links.find_by_partial_text('Next').click()

In [44]:
browser.quit()